The average starting salary of MBA graduates in US is provided in the file 08_MBA_Starting_Salary.xlsx.
Link: https://drive.google.com/drive/folders/1Jl8iDu7nGmrqCECbrLqmVafgwE5PYfiU

Use the variables below to predict the Avg Starting salary:

Type  
Enrollment  
Avg GMAT  
Resident Tuition, Fees  
Pct International  
Pct Female  
Pct Asian American  
Pct Minority  
Pct with job offers

1) Train a Decision tree by dropping  the rows with missing values 

2) Impute the missing values in each column using KNN imputer, and then train a model 

3) Compare the score of both the above models.  Use 5 fold CV score in both cases, with same model hyper-parameters (depth, etc)

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read and display the data file
df = pd.read_excel('/Users/riteshturlapaty/ai-ml-learning/AccelerateAI/7.DecisionTree/DailyQuiz/08_MBA_Starting_Salary.xlsx',sheet_name='MBA Data')
df.head(5)

,Fulltime Business Week Ranking,School Name,State,Type,Enrollment,Avg GMAT,"Resident Tuition, Fees",Pct International,Pct Female,Pct Asian American,Pct Minority,Pct with job offers,Avg starting base salary
0,1,University of Chicago,Illinois,Private,1144,713.0,97165.0,35.0,35.0,16.0,7.0,92.0,107091.0
1,2,Harvard University,Massachusetts,Private,1801,720.0,101660.0,33.0,38.0,NaN,NaN,94.0,124378.0
2,3,Northwestern University,Illinois,Private,1200,711.0,93918.0,34.0,36.0,25.0,13.0,95.0,108064.0
3,4,University of Pennsylvania,Pennsylvania,Private,1651,714.0,104410.0,44.0,36.0,7.8,9.0,89.0,112186.0
4,5,University of Michigan,Michigan,Public,898,706.0,80879.0,27.0,34.0,21.0,13.0,89.0,103608.0


In [3]:
# list out the columns
df.columns

Index(['Fulltime Business Week Ranking', 'School Name', 'State', 'Type',
       'Enrollment', 'Avg GMAT', 'Resident Tuition, Fees', 'Pct International',
       'Pct Female', 'Pct Asian American', 'Pct Minority',
       'Pct with job offers', 'Avg starting base salary'],
      dtype='object')

In [4]:
df.isnull().sum()

Fulltime Business Week Ranking    0
School Name                       0
State                             0
Type                              0
Enrollment                        0
Avg GMAT                          3
Resident Tuition, Fees            2
Pct International                 2
Pct Female                        2
Pct Asian American                7
Pct Minority                      4
Pct with job offers               3
Avg starting base salary          3
dtype: int64

In [5]:
# Create list of variables based on which prediction needs to be done
pred_var_list=['Type',
               'Enrollment', 
               'Avg GMAT', 
               'Resident Tuition, Fees',  
               'Pct International',
               'Pct Female', 
               'Pct Asian American', 
               'Pct Minority',
               'Avg starting base salary']

target='Avg starting base salary'

# 1) Train a Decision tree by dropping the rows with missing values

In [6]:
df1=df

In [7]:
# Create dummy values for categorical variables
df_model = df1[pred_var_list]
df_model = pd.get_dummies(df_model, columns=['Type'])

In [8]:
# Remove the rows with missing values
data_model_1=df_model.dropna()

# Compare the shape of the models
print('Original shape: ',df_model.shape)
print('Shape after pruning missing values rows: ',data_model_1.shape)

Original shape:  (70, 10)
Shape after pruning missing values rows:  (61, 10)


In [9]:
data_model_1.sample(2)

,Enrollment,Avg GMAT,"Resident Tuition, Fees",Pct International,Pct Female,Pct Asian American,Pct Minority,Avg starting base salary,Type_Private,Type_Public
38,293,662.0,37700.0,45.0,27.0,13.0,6.0,83614.0,0,1
2,1200,711.0,93918.0,34.0,36.0,25.0,13.0,108064.0,1,0


In [10]:
new_pred_var_list=data_model_1.columns

In [11]:
# Setup model
# Tune the parameters

tune_parm_space = {'min_samples_split':range(1, 20),
                   'max_depth':range(1, 20),
                   'min_samples_leaf':range(1, 20)
                  }

tree_reg_model = DecisionTreeRegressor(random_state=1)
tree_reg_model_cv_1 = GridSearchCV(tree_reg_model, tune_parm_space, cv=5)
tree_reg_model_cv_1.fit(data_model_1[new_pred_var_list], data_model_1[target])

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=1),
             param_grid={'max_depth': range(1, 20),
                         'min_samples_leaf': range(1, 20),
                         'min_samples_split': range(1, 20)})

In [12]:
tree_reg_model_cv_1.best_params_

{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}

# 2) Impute the missing values in each column using KNN imputer, and then train a model

In [13]:
# Assign to a new data frame in order to create two different models and compare them
df2=df

In [14]:
df_model.head(2)

,Enrollment,Avg GMAT,"Resident Tuition, Fees",Pct International,Pct Female,Pct Asian American,Pct Minority,Avg starting base salary,Type_Private,Type_Public
0,1144,713.0,97165.0,35.0,35.0,16.0,7.0,107091.0,1,0
1,1801,720.0,101660.0,33.0,38.0,NaN,NaN,124378.0,1,0


In [15]:
# 2. Use knn imputer for treating missing values
df_feed_knn=df_model
imputer = KNNImputer(n_neighbors=2)
df_knn = imputer.fit_transform(df_feed_knn)
data_model_2 = pd.DataFrame(df_knn, columns=new_pred_var_list)
data_model_2[target] = df_model[target]
print(data_model_2.shape)

(70, 10)


In [16]:
data_model_2.head(2)

,Enrollment,Avg GMAT,"Resident Tuition, Fees",Pct International,Pct Female,Pct Asian American,Pct Minority,Avg starting base salary,Type_Private,Type_Public
0,1144.0,713.0,97165.0,35.0,35.0,16.0,7.0,107091.0,1.0,0.0
1,1801.0,720.0,101660.0,33.0,38.0,7.9,11.0,124378.0,1.0,0.0


In [18]:
# remove rows, where target variable is missing
df_knn_imputed = data_model_2[~ data_model_2['Avg starting base salary'].isnull()]
df_knn_imputed.shape

(67, 10)

In [23]:
df_knn_imputed.head(2)

,Enrollment,Avg GMAT,"Resident Tuition, Fees",Pct International,Pct Female,Pct Asian American,Pct Minority,Avg starting base salary,Type_Private,Type_Public
0,1144.0,713.0,97165.0,35.0,35.0,16.0,7.0,107091.0,1.0,0.0
1,1801.0,720.0,101660.0,33.0,38.0,7.9,11.0,124378.0,1.0,0.0


In [24]:
df_knn_imputed.isna().any()

Enrollment                  False
Avg GMAT                    False
Resident Tuition, Fees      False
Pct International           False
Pct Female                  False
Pct Asian American          False
Pct Minority                False
Avg starting base salary    False
Type_Private                False
Type_Public                 False
dtype: bool

In [25]:
tune_parm_space = {'min_samples_split':range(1, 20),
                   'max_depth':range(1, 20),
                   'min_samples_leaf':range(1, 20)
                  }

tree_reg_model = DecisionTreeRegressor(random_state=1)
tree_reg_model_cv_2 = GridSearchCV(tree_reg_model, tune_parm_space, cv=5)
tree_reg_model_cv_2.fit(df_knn_imputed[new_pred_var_list], df_knn_imputed[target])

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=1),
             param_grid={'max_depth': range(1, 20),
                         'min_samples_leaf': range(1, 20),
                         'min_samples_split': range(1, 20)})

In [26]:
tree_reg_model_cv_1.best_params_

{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [30]:
score_1 = tree_reg_model_cv_1.score(data_model_1[new_pred_var_list], data_model_1[target])
score_2 = tree_reg_model_cv_2.score(df_knn_imputed[new_pred_var_list], df_knn_imputed[target])

rmse_1 = mean_squared_error(data_model_1[target], tree_reg_model_cv_1.predict(data_model_1[new_pred_var_list]), squared=False)
rmse_2 = mean_squared_error(df_knn_imputed[target], tree_reg_model_cv_2.predict(df_knn_imputed[new_pred_var_list]), squared=False)

print(f"Model 1 R square is {np.round(score_1 * 100, 2)} and RMSE is {np.round(rmse_1, 2)}")
print(f"Model 2 R square is {np.round(score_2 * 100, 2)} and RMSE is {np.round(rmse_2, 2)}")

Model 1 R square is 96.22 and RMSE is 2965.76
Model 2 R square is 99.86 and RMSE is 605.24


# Summary

1. For Model 1, R sqaure is 96.22 while RMSE is 2965.76
2. For Model 2, R sqaure is 99.86 while RMSE is 605.24. It is in model 2 where we used KNN Imputer.
3. Model parameter for both the models is {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
4. As evident, there is marginal improvement in Model 2 when compared to Model 1